# Example 2. Finite Element 1D

In [16]:
# Activate the package Jalerkin (located at top directory, "..")
using Pkg
JALERKIN_DIR = ".."
Pkg.activate(JALERKIN_DIR)

  Activating project at `~/src/julia/Jalerkin`


In [17]:
# 'Revise' allows modules to be autoreloaded if they are modified
# It should be used *only while Jalerkin is being developed* (it slows down the code)
using Revise  

# Load Jalerkin package
using Jalerkin

In [18]:
# Test Quadrature: 

# a) compute $\int_0^1 x^3 dx$ using Simpson's quad. rule
nodes = [0, 0.5, 1]
weights = [1/6, 2/3, 1/6]
simpson_qr = Quadrature{3}(nodes, weights)
println(quad(simpson_qr, x->x^3))
@assert quad(simpson_qr, x->x^3) ≈ 1/4

# b) compute $\int_{-1}^1 x^6 dx$ using Gaussian quad. rule
gauss_qr = GaussianQuadrature(4) # Exact up to order 2*4-1 = 7
print(quad(gauss_qr, x->x^7))
@assert quad(gauss_qr, x->x^7) ≈ 0

0.25
0.0

In [19]:
# Define a mesh in the interval [0, 1]
a, b = 0, 1
ncells = 2
mesh = Mesh1D(a, b, ncells)

Mesh{StaticArrays.SVector{2, Int64}, StaticArrays.SVector{1, Float64}}(StaticArrays.SVector{2, Int64}[[1, 2], [2, 3]], StaticArrays.SVector{1, Float64}[[0.0], [0.5], [1.0]])

In [20]:
# Define P1-Lagrange Finite Element space on the mesh with Simpson quad rule
fe = FiniteElement(mesh, Lagrange, 1, quad_rule=simpson_qr)
println("Order: $(get_order(fe))\n")
println("Mesh: $(get_mesh(fe))\n")
println("Quad. rule: $(get_quad_rule(fe)))\n")

Order: 1

Mesh: Mesh{StaticArrays.SVector{2, Int64}, StaticArrays.SVector{1, Float64}}(StaticArrays.SVector{2, Int64}[[1, 2], [2, 3]], StaticArrays.SVector{1, Float64}[[0.0], [0.5], [1.0]])

Quad. rule: Quadrature{3}([0.0, 0.5, 1.0], [0.16666666666666666, 0.6666666666666666, 0.16666666666666666]))



In [21]:
# Define P2-Lagrange Finite Element with default gaussian quad rule
fe = FiniteElement(mesh, Lagrange, 2)
println("Order: $(get_order(fe))\n")
println("Mesh: $(get_mesh(fe))\n")
println("Quad. rule: $(get_quad_rule(fe)))\n")

Order: 2

Mesh: Mesh{StaticArrays.SVector{2, Int64}, StaticArrays.SVector{1, Float64}}(StaticArrays.SVector{2, Int64}[[1, 2], [2, 3]], StaticArrays.SVector{1, Float64}[[0.0], [0.5], [1.0]])

Quad. rule: Quadrature{4}([-0.8611363115940526, -0.3399810435848563, 0.3399810435848563, 0.8611363115940526], [0.34785484513745385, 0.6521451548625462, 0.6521451548625462, 0.34785484513745385]))



In [22]:
# P1 basis function: evaluation on quadrature points (in reference element)
fe = FiniteElement(mesh, Lagrange, 1)
get_phi(fe)

2-element Vector{StaticArrays.SVector{2, Float64}}:
 [0.7886751345948129, 0.21132486540518708]
 [0.21132486540518708, 0.7886751345948129]

In [23]:
# P2 basis function: evaluation on quadrature points (in reference element)
fe = FiniteElement(mesh, Lagrange, 2)
get_phi(fe)

3-element Vector{StaticArrays.SVector{4, Float64}}:
 [-0.8013460293699308, -0.22778407679095214, 0.11219696679390419, 0.05979028222412169]
 [0.25844425285419076, 0.8844128900029521, 0.8844128900029521, 0.25844425285419076]
 [-0.05979028222412169, -0.11219696679390419, 0.22778407679095214, 0.8013460293699308]

In [24]:
element_index = 1
JxW = get_JxW(fe, element_index)

4-element StaticArrays.SVector{4, Float64} with indices SOneTo(4):
 0.08696371128436346
 0.16303628871563655
 0.16303628871563655
 0.08696371128436346

In [25]:
#
# Compute the integral at element 1 of phi_i*phi_j
#

fe = FiniteElement(mesh, Lagrange, 1)
phi = get_phi(fe) # Vector of basis functions (evaluated at each quadrature node)
element_index = 1
JxW = get_JxW(fe, element_index)  # Vector "Jacobian_matrix*quadrature_weights_vector"

i, j = 1, 2
println("phi_i evaluated on quad points: ", phi[i])
println("phi_j evaluated on quad points: ", phi[j])
println("JxW_element on quad points:     ", JxW)
numerical_integral = sum(JxW.*phi[i].*phi[j])
print("int_element(phi_i*phi_j) ≃ sum_q(JxW[q]*phi_i[q]*phi_j[q]): ", numerical_integral)

phi_i evaluated on quad points: [0.7886751345948129, 0.21132486540518708]
phi_j evaluated on quad points: [0.21132486540518708, 0.7886751345948129]
JxW_element on quad points:     [0.25, 0.25]
int_element(phi_i*phi_j) ≃ sum_q(JxW[q]*phi_i[q]*phi_j[q]): 0.08333333333333331

## Build mass mattrix
$$
M_{ij} = \int_\Omega \phi_i(x) \phi_j(x), \quad i,j=1,...,N_{\text{dof}}
$$

**Algorithm** for each element, $K\in {\cal T}_h$:
1. **Compute local matrix**,    
$$
M^K_{ij} = \int_K \phi_i(x) \phi_j(x), \quad i,j=1,\dots,N^K_{\text{dof}}
$$
where $N^K_{\text{dof}}$ is the number of degrees of freedom in element $K$ (for instance, in $P_1-Lagrange$ elements, $N^K=3$).
2. **Add to the global matrix** the these local contributions:
$$
M_{IJ} = M_{IJ} + M^K_{i,j},  \quad i,j=1,...,N^K_{\text{dof},}
$$
where notation $I,J \in \{1,\dots,N_{\text dofs}\}$ ($N_{\text dofs}$ is the total number of degrees of freedom in the mesh) represent the global indices  of the the degrees of freedom whose local indices (in element $K$) are repectively $i,j \in \{1,\dots,N^K_{\text dofs}\}$

In [26]:
#
# Bulid **local mass matrix** in one element
#
n = num_local_dofs(fe)
Me = Matrix(undef, n, n)
id_element = 1
JxW = get_JxW(fe, id_element)
for i=1:n
    for j=1:n
        Me[i,j] = sum(@. JxW*phi[i]*phi[j])
    end
end
Me

2×2 Matrix{Any}:
 0.166667   0.0833333
 0.0833333  0.166667

In [27]:
#
# Build **global mass matrix** (version 1)
#
using SparseArrays
M = spzeros((num_global_dofs(fe), num_global_dofs(fe)))
n = num_local_dofs(fe)
M_e = Matrix(undef, n, n)
for id_elem = 1:num_elements(fe)
    # 1) Compute local matrix
    JxW = get_JxW(fe, id_element)
    for i=1:n
        for j=1:n
            M_e[i,j] = sum(@. JxW*phi[i]*phi[j])
        end
    end
    # 2) Add local contributions to global matrix
    dof_indices = get_dof_indices(fe, id_elem)
    for i=1:n
        I = dof_indices[i]
        for j=1:n
            J = dof_indices[j]
            M[I,J] += M_e[i,j]
        end
    end
end
M

3×3 SparseMatrixCSC{Float64, Int64} with 7 stored entries:
 0.166667   0.0833333   ⋅ 
 0.0833333  0.333333   0.0833333
  ⋅         0.0833333  0.166667

In [28]:
#
# Build **global mass matrix** (version 2) 
#
using SparseArrays
M = spzeros((num_global_dofs(fe), num_global_dofs(fe)))
n = num_local_dofs(fe)
M_e = Matrix(undef, n, n)
for id_elem = 1:num_elements(fe)
    # 1) Compute local matrix
    JxW = get_JxW(fe, id_element)    
    for i=1:n
        for j=1:n
            M_e[i,j] = sum(@. JxW*phi[i]*phi[j])
        end
    end
    # 2) Add local contributions to global matrix
    dof_indices = get_dof_indices(fe, id_elem)
    add_matrix(M, M_e, dof_indices)
end
M

3×3 SparseMatrixCSC{Float64, Int64} with 7 stored entries:
 0.166667   0.0833333   ⋅ 
 0.0833333  0.333333   0.0833333
  ⋅         0.0833333  0.166667

In [29]:
#
# Build **global mass matrix** (version 3) 
#
using SparseArrays
M = spzeros((num_global_dofs(fe), num_global_dofs(fe)))
n = num_local_dofs(fe)
M_e = Matrix(undef, n, n)
for id_elem = 1:num_elements(fe)
    # 1) Compute local matrix
    integrate(M_e, (i, j) -> phi[i].*phi[j], id_elem, fe)
    # 2) Add local contributions to global matrix
    dof_indices = get_dof_indices(fe, id_elem)
    add_matrix(M, M_e, dof_indices)
end
M

3×3 SparseMatrixCSC{Float64, Int64} with 7 stored entries:
 0.166667   0.0833333   ⋅ 
 0.0833333  0.333333   0.0833333
  ⋅         0.0833333  0.166667